# View Data

In [1]:
from google.colab import drive
import pandas as pd
from tqdm import tqdm_notebook

## Examine data
### List of Excluded Individuals and Entities (LEIE)
https://oig.hhs.gov/exclusions/exclusions_list.asp

In [2]:
drive.mount('/content/drive')
leie = pd.read_csv('/content/drive/MyDrive/ISYE 6740 Project Files/LEIE_UPDATED.csv')

Mounted at /content/drive


<ipython-input-2-b5fdfd90f782>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  leie = pd.read_csv('/content/drive/MyDrive/ISYE 6740 Project Files/LEIE_UPDATED.csv')


In [3]:
# Take a look at the first few rows
leie.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,NaN,NaN,NaN,"#1 MARKETING SERVICE, INC",OTHER BUSINESS,SOBER HOME,NaN,0,NaN,239 BRIGHTON BEACH AVENUE,BROOKLYN,NY,11235,1128a1,20200319,0,0,NaN
1,NaN,NaN,NaN,"1 BEST CARE, INC",OTHER BUSINESS,HOME HEALTH AGENCY,NaN,0,NaN,"2161 UNIVERSITY AVENUE W, STE",SAINT PAUL,MN,55114,1128b5,20230518,0,0,NaN
2,NaN,NaN,NaN,101 FIRST CARE PHARMACY INC,OTHER BUSINESS,PHARMACY,NaN,1972902351,NaN,"C/O 609 W 191ST STREET, APT D",NEW YORK,NY,10040,1128b8,20220320,0,0,NaN
3,NaN,NaN,NaN,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN
4,NaN,NaN,NaN,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN


- There are a lot of NaN values for all the segments of name (first, middle, last), UPIN, and DoB.
- There are a lot of 0s for NPI, which might be a problem considering that this particular field is how we're planning to join the LEIE to other data...

In [4]:
# Take a look at the distribution of null columns.
# However, some numerical columns (NPI, REINDATE, WAIVERDATE) need additional scrutiny since it seems that they are inputted as 0s.
leie['NPI'] = leie['NPI'].replace(0,pd.NA)
leie['REINDATE'] = leie['REINDATE'].replace(0,pd.NA)
leie['WAIVERDATE'] = leie['WAIVERDATE'].replace(0,pd.NA)
leie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81471 entries, 0 to 81470
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LASTNAME    78143 non-null  object 
 1   FIRSTNAME   78142 non-null  object 
 2   MIDNAME     57497 non-null  object 
 3   BUSNAME     3326 non-null   object 
 4   GENERAL     81471 non-null  object 
 5   SPECIALTY   77344 non-null  object 
 6   UPIN        6134 non-null   object 
 7   NPI         7923 non-null   object 
 8   DOB         77281 non-null  float64
 9   ADDRESS     81462 non-null  object 
 10  CITY        81470 non-null  object 
 11  STATE       81471 non-null  object 
 12  ZIP         81471 non-null  int64  
 13  EXCLTYPE    81471 non-null  object 
 14  EXCLDATE    81471 non-null  int64  
 15  REINDATE    0 non-null      object 
 16  WAIVERDATE  4 non-null      object 
 17  WVRSTATE    5 non-null      object 
dtypes: float64(1), int64(2), object(15)
memory usage: 11.2+ MB


- REINDATE has no non-null values, which makes sense since LEIE does not include any providers that have been re-instated. Thus, we can probably delete those three columns.
- ...We have a lot of Null NPIs, which will become a problem. Looks like we might have to consder alternative ethods to joining.

In [5]:
# Are the providers in here unique (i.e. Does each NPI only appear once.)
leie['NPI'].value_counts().head()

,count
NPI,
1508904798,3
1225072028,3
1801839139,3
1427503093,2
1992978571,2


In [6]:
leie[leie['NPI']==1801839139]

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
16166,CHRISTENSEN,JOHN,PETER,NaN,"PHYSICIAN (MD, DO)",GENERAL PRACTICE,K9897A,1801839139,19510925.0,2900 N FLAGER DRIVE,WEST PALM BEACH,FL,33407,1128b4,20131120,<NA>,<NA>,NaN
16167,CHRISTENSEN,JOHN,PETER,NaN,"PHYSICIAN (MD, DO)",GENERAL PRACTICE,NaN,1801839139,19510925.0,"568 NE 255TH STREET, #D63217",CROSS CITY,FL,32628,1128a2,20190220,<NA>,<NA>,NaN
16168,CHRISTENSEN,JOHN,NaN,NaN,"PHYSICIAN (MD, DO)",GENERAL PRACTICE,I53515,1801839139,19510925.0,P O BOX 019120,MIAMI,FL,33101,1128a1,20170518,<NA>,<NA>,NaN


- NPI values are NOT unique; separate entry created for each violation?
- One of the rows had blank MIDNAME and UPIN; can we fill in NPIs potentially? (No)

In [7]:
# Group by first name, last name, and state, then count unique NPI values
npi_counts = leie.groupby(['LASTNAME', 'FIRSTNAME', 'STATE', 'DOB'])['NPI'].nunique(dropna=False)
npi_counts[npi_counts > 1]

,,,,NPI
LASTNAME,FIRSTNAME,STATE,DOB,
BALTAZAR,RODNEY,MD,19621222.0,2
BAUER,KURT,PA,19530416.0,2
BERNHARD,LARRY,MD,19550619.0,2
CARTER,CORNELIUS,OK,19640703.0,2
CHIRBAN,ANGELO,AZ,19490422.0,2
COZOLINO,CLIFFORD,NY,19590105.0,2
CRESPIN,JOSE,FL,19550516.0,2
DOSS,HEATHER,OK,19740708.0,2
FERNANDEZ,EDUARDO,FL,19510818.0,2


In [8]:
leie[(leie['LASTNAME']=='MCCUSKER') & (leie['FIRSTNAME']=='CHARLES')]

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
48790,MCCUSKER,CHARLES,F,NaN,"PHYSICIAN (MD, DO)",PSYCHIATRY,NaN,1104028216,19520215.0,2689 S HIGHLAND DRIVE,SALT LAKE CITY,UT,84106,1128Aa,20141114,<NA>,<NA>,NaN
48791,MCCUSKER,CHARLES,NaN,NaN,PSYCHOLOGIC PRACTICE,PSYCHOLOGY,NaN,<NA>,19520215.0,"1627 S FOOTHILL DR, #10",SALT LAKE CITY,UT,84108,1128b4,20020520,<NA>,<NA>,NaN


- Seems possible, certainly, but on other thought is it really necessary...? Since what we really want is a list of providers in the LEIE...

In [9]:
leie[(leie['NPI'].isnull()) & (leie['LASTNAME'].notnull())]

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
3328,AAKER,DEBHANNA,NaN,NaN,EMPLOYEE - PRIVATE S,HOME HEALTH AGENCY,NaN,<NA>,19820311.0,2006 OAK ST,GRAND FORKS,ND,58201,1128a1,20240820,<NA>,<NA>,NaN
3329,AALDERS,NANCY,BLOCK,NaN,HOSPITAL,NURSE/NURSES AIDE,NaN,<NA>,19640305.0,9527 SOUTH KENTON,OAK LAWN,IL,60453,1128b4,20030820,<NA>,<NA>,NaN
3330,AALTONEN,NICKOLAS,A,NaN,IND- LIC HC SERV PRO,PHARMACY TECHNICIAN,NaN,<NA>,19880123.0,32 W PACIFIC AVE,SPOKANE,WA,99201,1128a4,20120419,<NA>,<NA>,NaN
3331,AAMIR,MUHAMMAD,SOHAIL,NaN,BUS OWNER/EXEC,HOME HEALTH AGENCY,NaN,<NA>,19700910.0,3956 COLUMBIA COURT,BLOOMFIELD HILLS,MI,48302,1128a1,20170220,<NA>,<NA>,NaN
3332,AARON,ALINA,A,NaN,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,NaN,<NA>,19870209.0,3325 N ROSE CIRCLE DRIVE,PHOENIX,AZ,85018,1128b4,20171019,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81466,ZWOLINSKI,AMBER,DAWN,NaN,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,NaN,<NA>,19870529.0,1238 PULASKI AVENUE,COAL TOWNSHIP,PA,17866,1128a1,20160519,<NA>,<NA>,NaN
81467,ZYDERVELD,MARY,ANN,NaN,PHARMACY,TECHNICIAN,NaN,<NA>,19520825.0,1358 MORNING GLORY TURN,RUCKERSVILLE,VA,22962,1128b4,20090219,<NA>,<NA>,NaN
81468,ZYLSTRA,CHRISTEENA,LYNN,NaN,NURSING PROFESSION,NURSE/NURSES AIDE,NaN,<NA>,19670724.0,1816 NE 49TH STREET,KANSAS CITY,MO,64118,1128b4,20100318,<NA>,<NA>,NaN
81469,ZYLSTRA,JAMES,PETER,NaN,REHAB FACILITY - GEN,EMPLOYEE,NaN,<NA>,19550420.0,"2305 MINNESOTA BLVD, S E",ST CLOUD,MN,56304,1128a2,20040920,<NA>,<NA>,NaN


- This has not yielded successfull results...god this is going to be a pain later on.

In [10]:
leie['SPECIALTY'].unique()

array(['SOBER HOME', 'HOME HEALTH AGENCY', 'PHARMACY', nan,
       'DME - OXYGEN', 'MARKETING FIRM', 'TRANSPORTATION CO',
       'DME - ORTHOTICS', 'DME - GENERAL', 'GENERAL PRACTICE',
       'MENTAL/BEHAVIORAL HE', 'COMM MNTL HLTH CNTR', 'CONSULTING FIRM',
       'ALLIED HEALTH RELATE', 'LAB - CLINICAL', 'PHYS THERAPY PROVIDE',
       'CLINIC', 'DME - PROSTHETICS', 'COUNSELING CENTER',
       'DME - HOME INFUSION', 'AMBULANCE COMPANY', 'CHIROPRACTIC PRACT',
       'OPHTHALMOLOGY', 'DENTAL PRACTICE', 'DME - DIETARY SUPPLI',
       'RESEARCH COMPANY', 'ALLERGIST/IMMUNOLOGY', 'HOSPITAL',
       'INTERNAL MEDICINE', 'PSYCHOLOGIC PRACTICE',
       'COMMUNITY HLTH CTR (', 'DME - HEARING AID', 'OPTOMETRIC PRACTICE',
       'RADIOLOGY', 'LABORATORY', 'FAMILY PRACTICE', 'CHIROPRACTIC',
       'DRUG COMPANY/SUPLIER', 'SUBSTANCE ABUSE REHA', 'LABORATORY-IDTF',
       'SKILLED NURSING FAC', 'CARDIOLOGY', 'DME - WOUND CARE KIT',
       'DME - UROLOGICAL SUP', 'GENERAL PRACTICE/FP',
       'MANUF/L

- A lot of those categories are healthcare-related, but quite a few aren't. ('MARKETING FIRM', 'ACCOUNTING FIRM', 'LOCAL GOV'T', etc.)

In [11]:
# Find the NPI null rate for each specialty and order by lowest to highest
npi_null_rate_by_specialty = leie.groupby('SPECIALTY')['NPI'].apply(lambda x: x.isnull().sum() / len(x))
npi_null_rate_by_specialty = npi_null_rate_by_specialty.sort_values()
npi_null_rate_by_specialty.head(20)

,NPI
SPECIALTY,
NATUROPATHY,0.000000
INFECTIOUS DISEASE,0.000000
PROCTOLOGY,0.000000
PAIN MANAGEMENT,0.027624
PHYSICIAN ASSISTANT,0.139665
NURSE PRACTITIONER (,0.177665
RHEUMATOLOGY,0.214286
EMERGENCY MEDICINE,0.264957
FAMILY PRACTICE,0.275253


- Damn there's a lot of providers with missing NPIs...

In [12]:
# Investigating EXCLTYPE
leie['EXCLTYPE'].value_counts()

,count
EXCLTYPE,
1128b4,32631
1128a1,24902
1128a2,7916
1128a3,5614
1128a4,3472
1128b14,2199
1128b8,1497
1128b1,926
1128b5,813


Info regarding types of exclusions can be found here:
- https://oig.hhs.gov/exclusions/authorities.asp
- https://oig.hhs.gov/faqs/exclusions-faq/

Generally:
- **1128** - The scope of an exclusion under section 1128 of the Act is from all **Federal health care programs**, as defined in 42 CFR 1001.2. Federal health care programs include Medicare, Medicaid, and all other plans and programs that provide health benefits funded directly or indirectly by the United States (other than the Federal Employees Health Benefits Plan).
- **1156** - The scope of an exclusion under section 1156 of the Act is from **Medicare and all State health care programs** as defined in section 1128(h) of the Act. Exclusions under section 1156 of the Act *do not reach other Federal programs*.

In [13]:
# Looking only at rows with non-null NPIs, visualize the top specialties.
leie[leie['NPI'].notnull()]['SPECIALTY'].value_counts().head(20)

,count
SPECIALTY,
NURSE/NURSES AIDE,779
GENERAL PRACTICE,774
FAMILY PRACTICE,574
INTERNAL MEDICINE,534
CHIROPRACTIC,415
COUNSELOR,358
DENTIST,326
PHARMACIST,293
PSYCHIATRY,242


In [14]:
# Looking only at rows with non-null NPIs, visualize the top states.
leie[leie['NPI'].notnull()]['STATE'].value_counts().head(20)

,count
STATE,
CA,1023
FL,724
NY,648
OH,584
TX,481
PA,373
MI,227
IL,220
WV,202


- Maybe take a look at number of Medicare recipients?

In [15]:
# Find min and max excldates
print(leie['EXCLDATE'].min())
print(leie['EXCLDATE'].max())

19770701
20250220


### Opioids List

In [20]:
opioids = pd.read_excel('/content/drive/MyDrive/ISYE 6740 Project Files/Opioid_Drug_Lists_CY2018.xlsx')
opioids.head()

,Medicare Part D CY 2018 Opioids,Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN
1,NaN,Drug Name,Generic Name
2,NaN,ABSTRAL,FENTANYL CITRATE
3,NaN,ACETAMIN-CAFF-DIHYDROCODEINE,ACETAMINOPHEN/CAFF/DIHYDROCOD
4,NaN,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE


In [17]:
opioids.tail()

,Medicare Part D CY 2018 Opioids,Unnamed: 1,Unnamed: 2
89,*,ZOHYDRO ER,HYDROCODONE BITARTRATE
90,NaN,Note:,NaN
91,NaN,* Long-acting opioid.,NaN
92,NaN,# Drug name includes NDC identifiers from the ...,NaN
93,NaN,as well as NDC identifiers not included in the...,NaN



```
* Long-acting opioid.
# Drug name includes NDC identifiers from the Prescriber Drug Category List for opioids
as well as NDC identifiers not included in the Prescriber Drug Category List for opioids.
```


In [21]:
opioids = opioids.iloc[2:90]
opioids

,Medicare Part D CY 2018 Opioids,Unnamed: 1,Unnamed: 2
2,NaN,ABSTRAL,FENTANYL CITRATE
3,NaN,ACETAMIN-CAFF-DIHYDROCODEINE,ACETAMINOPHEN/CAFF/DIHYDROCOD
4,NaN,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE
5,NaN,ACTIQ,FENTANYL CITRATE
6,*,ARYMO ER,MORPHINE SULFATE
...,...,...,...
85,NaN,VICODIN HP,HYDROCODONE/ACETAMINOPHEN
86,NaN,XODOL 10-300,HYDROCODONE/ACETAMINOPHEN
87,NaN,XODOL 7.5-300,HYDROCODONE/ACETAMINOPHEN
88,*,XTAMPZA ER,OXYCODONE MYRISTATE


In [23]:
opioids.columns = ['Addenda','Drug Name', 'Generic Name']
opioids['Long-acting opioid'] = opioids['Addenda'].apply(lambda x: 1 if '*' in x else 0)
opioids['NDC'] = opioids['Addenda'].apply(lambda x: 1 if '#' in x else 0)
opioids

,Addenda,Drug Name,Generic Name,Long-acting opioid,NDC
2,,ABSTRAL,FENTANYL CITRATE,0,0
3,,ACETAMIN-CAFF-DIHYDROCODEINE,ACETAMINOPHEN/CAFF/DIHYDROCOD,0,0
4,,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE,0,0
5,,ACTIQ,FENTANYL CITRATE,0,0
6,*,ARYMO ER,MORPHINE SULFATE,1,0
...,...,...,...,...,...
85,,VICODIN HP,HYDROCODONE/ACETAMINOPHEN,0,0
86,,XODOL 10-300,HYDROCODONE/ACETAMINOPHEN,0,0
87,,XODOL 7.5-300,HYDROCODONE/ACETAMINOPHEN,0,0
88,*,XTAMPZA ER,OXYCODONE MYRISTATE,1,0


In [24]:
opioids = opioids.drop(columns=['Addenda'])

In [25]:
leie.to_csv('/content/drive/MyDrive/ISYE 6740 Project Files/leie_cleaned.csv', index=False)
opioids.to_csv('/content/drive/MyDrive/ISYE 6740 Project Files/opioids_cleaned.csv', index=False)